In [2]:
! pip install pygad

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [3]:
import numpy as np
from dgamod import *
import csv
import pygad
import sys
import time
import os
import configparser

Queremos probar el desempeño del genético para optimizar usando una acción por sitio, testeo primero la función que genera las matrices de acciones

In [7]:
matrices = one_field_actions(bmax=100,nh=10)
#print(matrices)
!mkdir test

Veo qué tengo que modificar del script que usa las otras

In [11]:
# get parameters from config file
# thisfolder = os.path.dirname(os.path.abspath(__file__))
# initfile = os.path.join(thisfolder, str(sys.argv[1]))
# print(initfile)

initfile = 'test.ini' #dejo algun config file para testear

config = configparser.ConfigParser()
config.read(initfile)


# system parameters
n = config.getint("system_parameters", "n")
dt = config.getfloat("system_parameters", "dt")
b = config.getfloat("system_parameters", "b")
speed_fraction = config.getfloat("system_parameters", "speed_fraction") # fraction of qsl speed if loc based fitness
max_optimization_time = config.getint("system_parameters", "max_optimization_time")

# aca genero las acciones
acciones = one_field_actions(b, n)  
props = gen_props(acciones, n, b, dt)

# genetic algorithm parameters
num_generations = config.getint("ga_initialization", "num_generations")
num_genes = config.getint("ga_initialization", "num_genes")
sol_per_pop = config.getint("ga_initialization", "sol_per_pop")
fidelity_tolerance = config.getfloat("ga_initialization", "fidelity_tolerance")
saturation = config.getint("ga_initialization", "saturation")
reward_decay = config.getfloat("ga_initialization", "reward_decay")

# crossover and parent selection

num_parents_mating = config.getint("parent_selection", "num_parents_mating")
parent_selection_type = config.get("parent_selection", "parent_selection_type")
keep_elitism = config.getint("parent_selection", "keep_elitism")
crossover_type = config.get("crossover", "crossover_type")
crossover_probability = config.getfloat("crossover", "crossover_probability")

# mutation
mutation_probability = config.getfloat("mutation", "mutation_probability")
mutation_num_genes = config.getint("mutation", "mutation_num_genes")

gene_space = np.arange(0, n, 1)
gene_type = int

stop_criteria = ["saturate_" + str(saturation)]  # , 'reach_'+str(fidelity_tolerance)]

population_histograms = config.getboolean("saving","population_histograms")
dirname = config.get("saving", "directory")
n_samples = config.getint("saving", "n_samples")

filename = dirname + "/nvsmaxfid.dat"

# call construction functions
on_generation = generation_func_constructor(
    generation_func, [props, fidelity_tolerance, dirname, population_histograms]
)

fidelity_args = [props,speed_fraction, max_optimization_time]#,fidelity_tolerance,reward_decay]
fitness_func = fitness_func_constructor(localization_based, fidelity_args)
mutation_type = "swap"


with open(filename, "a") as f:
    for i in range(n_samples):
        writer = csv.writer(f, delimiter=" ")

        solutions_fname = "{}/act_sequence_n{}_sample{}.dat".format(dirname, n, i)
        # fitness_history_fname = dirname + '/fitness_history_sample'+ str(i) + '.dat'

        t1 = time.time()

        initial_instance = pygad.GA(
            num_generations=num_generations,
            num_parents_mating=num_parents_mating,
            fitness_func=fitness_func,
            sol_per_pop=sol_per_pop,
            num_genes=num_genes,
            parent_selection_type=parent_selection_type,
            keep_elitism=keep_elitism,
            gene_space=gene_space,
            gene_type=gene_type,
            crossover_type=crossover_type,
            crossover_probability=crossover_probability,
            mutation_type=mutation_type,
            on_generation=on_generation,
            mutation_num_genes=mutation_num_genes,
            stop_criteria=stop_criteria,
            save_solutions=False
        )

        initial_instance.run()

        t2 = time.time()
        trun = t2 - t1

        maxg = initial_instance.generations_completed

        solution, solution_fitness, solution_idx = initial_instance.best_solution()

        evolution = time_evolution(solution, props, n, graph=False, filename=False)
        time_max_fidelity = np.argmax(evolution) * dt

        row = [
            n,
            i,
            format(fidelity(solution, props)),
            "{:.8f}".format(time_max_fidelity),
            maxg,
            "{:.8f}".format(trun),
        ]
        writer.writerow(row)
        actions_to_file(solution, solutions_fname, "w")

Propagacion de autoestados: correcta
Generation 1
Solution:  [7 9 6 9 5 6 3 8 9 8 5 7 1 1 7 0 0 9 8 7 3 3 2 0 4 0 0 4 1 2 2 4 8 9 7 0 6
 1 9 1 2 4 1 4 6 6 3 5 8 0] Fidelity:  0.1647985411428337 Time:  39 Fitness:  0.005735528880210453
Generation 2
Solution:  [9 5 3 4 5 5 4 6 6 9 7 9 9 5 1 6 7 1 1 9 8 7 1 8 2 1 4 1 4 4 0 7 7 5 7 6 9
 3 4 8 1 2 4 6 6 7 0 6 1 6] Fidelity:  0.35658838522321556 Time:  48 Fitness:  0.024761793854227363
Generation 3
Solution:  [8 3 3 3 3 7 8 6 4 0 7 9 5 0 8 8 7 8 7 2 9 7 1 0 8 2 0 9 2 4 8 5 4 5 9 4 6
 1 5 3 5 4 4 3 3 3 2 1 8 2] Fidelity:  0.5727991110045932 Time:  50 Fitness:  0.033436260160861674
Generation 4
Solution:  [6 3 4 9 7 4 7 7 9 8 7 8 9 7 8 0 0 9 0 2 9 3 2 3 2 9 3 9 4 3 2 4 4 1 5 5 0
 6 1 1 0 1 4 6 0 0 5 5 3 2] Fidelity:  0.7440502459060988 Time:  41 Fitness:  0.11126514035604718
Generation 5
Solution:  [4 2 6 9 5 4 6 4 5 5 9 8 9 7 7 0 0 8 9 9 8 0 9 0 1 8 9 9 1 9 3 1 4 1 4 4 2
 2 5 1 3 0 2 2 5 2 7 5 4 9] Fidelity:  0.8939900865516198 Time:  43 Fitn

KeyboardInterrupt: 